<a href="https://colab.research.google.com/github/ROzenSaglam/Face_detection/blob/main/image_detection_first_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install labelme tensorflow opencv-python matplotlib albumentations

In [ ]:
# 1. Collecting Images

import os
import time
import uuid
import cv2

print("Successfully imported!")

In [ ]:
import json
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf


In [ ]:
image_path = os.path.join('data','image')
number_images = 30

In [ ]:
images = tf.data.Dataset.list_files('/content/drive/MyDrive/data/image/*.jpg', shuffle = False)

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
#to read an image, decode it and reshape the tensor containing the pixel data:

def load_image(x):
  raw = tf.io.read_file(x)
  img = tf.image.decode_jpeg(raw)

  return img

In [ ]:
images = images.map(load_image)

In [ ]:
images.as_numpy_iterator().next()


In [ ]:
image_generator = images.batch(4).as_numpy_iterator()

In [ ]:
plt_img = image_generator.next()

In [ ]:
fig, ax = plt.subplots(ncols = 4, figsize=(20,20))
for idx, image in enumerate(plt_img):
  ax[idx].imshow(image)
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive"


In [ ]:
#moving matching labels:

for folder in ['train', 'test', 'valid']:
  for file in os.listdir(os.path.join('drive','MyDrive','data', folder, 'image')):
    filename = file.split('.')[0]+'.json'
    existing_filepath = os.path.join('drive','MyDrive','data', 'label', filename)
    if os.path.exists(existing_filepath):
      new_filepath = os.path.join('drive','MyDrive','data', folder, 'label', filename)
      os.replace(existing_filepath, new_filepath)

Apply Image Augmentation on Images and Labels

In [ ]:
import albumentations as alb

Augmentation Pipeline:

In [ ]:
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

augmentor = alb.Compose([alb.HorizontalFlip(p=0.5),
             alb.RandomBrightnessContrast(p=0.3),
             alb.RandomCrop(width=700, height=700)],
             bbox_params=alb.BboxParams(format='albumentations', label_fields = ['class_labels']))


Test an Image & Annotation

In [ ]:
img = cv2.imread(os.path.join('drive','MyDrive','data','train','image','27f557a6-850c-11ed-95e4-1e00f1349d90.jpg'))

with open(os.path.join('drive','MyDrive','data','train','label','27f557a6-850c-11ed-95e4-1e00f1349d90.json'), 'r') as f:
  label = json.load(f)


In [ ]:
type(img)

In [ ]:
img

In [ ]:
filee = '27f557a6-850c-11ed-95e4-1e00f1349d90.jpg'
new_file = (filee.split('.'))[0]
new_file
name = f'{new_file}.json'
name

In [ ]:
label['shapes'][0]['points']

take the coordinates as a list from the dict 'label'

In [ ]:
coords = [0,0,0,0]
coords[0] = label['shapes'][0]['points'][0][0]
coords[1] = label['shapes'][0]['points'][0][1]
coords[2] = label['shapes'][0]['points'][1][0]
coords[3] = label['shapes'][0]['points'][1][1]

In [ ]:
coords

normalize the coordinates by dividing x with width and y with height of the image

In [ ]:
type(img)

In [ ]:
coords = list(np.divide( coords, [1280,720,1280,720]))

In [ ]:
coords

In [ ]:
type(coords)

try augmentation pipeline

In [ ]:
type(img)

In [ ]:
aug = augmentor(image = img, bboxes=[coords], class_labels = ['face'])
cv2.rectangle(aug['image'],
              tuple(np.multiply(aug['bboxes'][0][:2], [700,700]).astype(int)),
              tuple(np.multiply(aug['bboxes'][0][2:], [700,700]).astype(int)),
              (0,255,0),2)

Build an Augmentation Pipeline for all images

In [ ]:
labels = []
type(labels)

In [ ]:
filee = '27f557a6-850c-11ed-95e4-1e00f1349d90.jpg'
new_file = (filee.split('.'))[0]
new_file
name = f'{new_file}.json'
name

In [ ]:


for folder in ['train', 'test', 'valid']:
  for files in os.listdir(os.path.join('drive','MyDrive','data', folder, 'image')):
    img = cv2.imread(os.path.join('drive','MyDrive','data', folder,'image', files))
    label_path = os.path.join('drive','MyDrive','data', folder,'label', f'{(files.split("."))[0]}.json')
    coords = [0,0,0.000001,0.000001]
    if os.path.exists(label_path):
      with open(label_path, 'r') as f:
        label = json.load(f)
      coords[0] = label['shapes'][0]['points'][0][0]
      coords[1] = label['shapes'][0]['points'][0][1]
      coords[2] = label['shapes'][0]['points'][1][0]
      coords[3] = label['shapes'][0]['points'][1][1]
      coords = list(np.divide(coords, [1280, 720, 1280, 720]))

    try:
      for x in range(60):
        augmented = augmentor(image = img, bboxes = [coords], class_labels = ['face'])
        cv2.imwrite(os.path.join('drive','MyDrive','data','aug_data',folder,'image', f'{(files.split("."))[0]}_{x}.jpg'), augmented['image'])
        annotation = {}
        annotation['image'] = files
        if os.path.exists(label_path):
          if len(augmented['bboxes']) == 0:
            annotation['bbox'] = [0,0,0,0]
            annotation['class'] = 0
          else:
            annotation['bbox'] = augmented['bboxes'][0]
            annotation['class'] = 1
        else:
          annotation['bbox'] = [0,0,0,0]
          annotation['class'] = 0

        with open(os.path.join('drive','MyDrive','data','aug_data',folder,'label', f'{(files.split("."))[0]}_{x}.json'), 'w') as f:
          json.dump(annotation, f)
    except Exception as e:
      print(e)





Define a image resizing funtion for your images

In [ ]:
  def normalize_image(image):
    return tf.image.resize(image, (120,120))


In [ ]:
#to read an image, decode it and reshape the tensor containing the pixel data:

def load_image(x):
  raw = tf.io.read_file(x)
  img = tf.image.decode_jpeg(raw)

  return img

# Load the files,
#turn them into TensorSliceDataset
#and normalize the image size and color::

In [ ]:



try:
  train_images = tf.data.Dataset.list_files("/content/drive/MyDrive/data/aug_data/train/image/*.jpg", shuffle = False)
  train_images = train_images.map(load_image)
  train_images = train_images.map(normalize_image)
  train_images = train_images.map(lambda x: x/255)

except Exception as e :
  print(e)





In [ ]:

try:
  test_images = tf.data.Dataset.list_files("/content/drive/MyDrive/data/aug_data/test/image/*.jpg", shuffle = False)
  test_images = test_images.map(load_image)
  test_images = test_images.map(normalize_image)
  test_images = test_images.map(lambda x: x/255)

except Exception as e :
  print(e)



In [ ]:

try:
  valid_images = tf.data.Dataset.list_files("/content/drive/MyDrive/data/aug_data/valid/image/*.jpg", shuffle = False)
  valid_images = valid_images.map(load_image)
  valid_images = valid_images.map(normalize_image)
  valid_images = valid_images.map(lambda x: x/255)

except Exception as e :
  print(e)

In [ ]:
len(train_images),len(valid_images),len(test_images)

# Load the labels, return the class and bboxes data :

In [ ]:

def read_label(label_path):
  with open(label_path.numpy(), 'r', encoding= 'utf-8') as f:
    label = json.load(f)

  return [label['class']], label['bbox']


In [ ]:
read_label("/content/drive/MyDrive/data/aug_data/train/label/27f557a6-850c-11ed-95e4-1e00f1349d90_0.json")


In [ ]:
train_labels = tf.data.Dataset.list_files("/content/drive/MyDrive/data/aug_data/train/label/*.json", shuffle = False)
train_labels = train_labels.map(lambda x: tf.py_function(func = read_label, inp = [x], Tout = [tf.uint8, tf.float16]))

valid_labels = tf.data.Dataset.list_files("/content/drive/MyDrive/data/aug_data/valid/label/*.json", shuffle = False)
valid_labels = valid_labels.map(lambda x: tf.py_function(func = read_label, inp = [x], Tout = [tf.uint8, tf.float16]))

test_labels = tf.data.Dataset.list_files("/content/drive/MyDrive/data/aug_data/test/label/*.json", shuffle = False)
test_labels = test_labels.map(lambda x: tf.py_function(func = read_label, inp = [x], Tout = [tf.uint8, tf.float16]))


In [ ]:
test_labels.as_numpy_iterator().next()

In [ ]:
len(train_labels),len(valid_labels),len(test_labels)

COmbine Label and Image Samples

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(4000)
train = train.batch(8)
train = train.prefetch(4)

test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(900)
test = test.batch(8)
test = test.prefetch(4)

valid = tf.data.Dataset.zip((valid_images, valid_labels))
valid = valid.shuffle(800)
valid = valid.batch(8)
valid = valid.prefetch(4)

In [ ]:
data_samples = train.as_numpy_iterator()
res = data_samples.next()

fig, ax =plt.subplots(ncols = 4, figsize=(20,20))
for idx in range(4):
  sample_image = res[0][idx]
  sample_coords = res[1][1][idx]

  cv2.rectangle(sample_image,
                tuple(np.multiply(sample_coords[:2], [120,120]).astype(int)),
                tuple(np.multiply(sample_coords[2:],[120,120]).astype(int)),
                (255,0,0),2)

  ax[idx].imshow(sample_image)

#Build Deep Learning Model
With the combination of both Classification and Regression

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [ ]:
vgg = VGG16(include_top = False)

Our CNN

In [ ]:
def build_model():
  input_layer = Input(shape=(120,120,3))
  vgg = VGG16(include_top = False)(input_layer)

  #Classification Model:
  f1 = GlobalMaxPooling2D()(vgg)
  class1 = Dense(2048, activation= 'relu')(f1)
  class2 = Dense(1, activation = 'sigmoid')(class1)

  #Regression Model:
  f2 = GlobalMaxPooling2D()(vgg)
  regress1 = Dense(2048, activation = 'relu')(f2)
  regress2 = Dense(4, activation = 'sigmoid')(regress1)

  facetracker = Model(inputs= input_layer, outputs = [class2, regress2])
  return facetracker

In [ ]:
facetracker = build_model()
facetracker.summary()

In [ ]:
train.as_numpy_iterator().next()[1]

In [ ]:
X, y = train.as_numpy_iterator().next()

In [ ]:
X.shape

In [ ]:
y

In [ ]:
classes, coords = facetracker.predict(X)

In [ ]:
classes, coords

Define Lossesand Optimizers


In [ ]:
from tensorflow.keras.optimizers import Adam
Batches_per_epoch = len(train)
lr_decay = (1./0.75-1)/Batches_per_epoch
opt = tf.keras.optimizers.Adam(learning_rate = 0.01,
                                     decay = lr_decay)

In [ ]:
#Localization Loss:

def loc_loss(y_true, yhat):
  delta_coord = tf.reduce_sum(tf.square(y_true[:,:2]-yhat[:,:2]))

  h_true = y_true[:,3] - y_true[:,1]
  w_true = y_true[:,2] - y_true[:,0]

  h_pred = yhat[:,3] - yhat[:,1]
  w_pred = yhat[:,2] - yhat[:,0]

  delta_size = tf.reduce_sum(tf.square(w_true-w_pred) + (h_true-h_pred))
  return delta_coord + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = loc_loss

#Train Neural Network

In [ ]:
class Facetracker(Model):
  def __init__(self, eyetracker,  **kwargs):
    super().__init__(**kwargs)
    self.model = eyetracker
  def compile(self, opt, classloss, regressloss, **kwargs):
    super().compile(**kwargs)
    self.closs = classloss
    self.lloss = regressloss
    self.opt = opt
  def train_step(self, batch, **kwargs):
    X, y = batch
    with tf.GradientTape() as tape:
      #Make predictions:
      classes, coords = self.model(X, training = True)
      #Calculate the loss:
      batch_classloss = self.closs(y[0], classes)
      batch_locloss = self.lloss(tf.cast(y[1], tf.float32), coords)
      total_loss = batch_classloss*0.5 + batch_locloss
      #calculate the gradients:
      grad = tape.gradient(total_loss, self.model.trainable_variables)
      #loop through each variable:
      opt.apply_gradients(zip(grad, self.model.trainable_variables))

      return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_locloss}

  def test_step(self, batch, **kwargs):
    X, y = batch
    classes, coords = self.model(X, training = False)
    batch_classloss = self.closs(y[0], classes)
    batch_locloss = self.lloss(tf.cast(y[1], tf.float32), coords)
    total_loss = batch_classloss*0.5 + batch_locloss
    return {"total_val_loss":total_loss, "val_class_loss":batch_classloss, "val_regress_loss":batch_locloss}





In [ ]:
model = Facetracker(facetracker)

In [ ]:
model.compile(opt, classloss, loc_loss)

Train

In [ ]:
hist = model.fit(train.take(100), epochs = 10, validation_data = valid)

In [ ]:
hist.history

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_val_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()
